In [ ]:
pip install google-play-scraper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.3 MB/s eta 0:00:00


In [ ]:
from google_play_scraper import reviews, Sort
import pandas as pd

# Ambil 10000 review terbaru
app_id = 'co.id.bankbsi.superapp'  # ID aplikasi BYOND by BSI
result, _ = reviews(
    app_id,
    lang='id',          # Bahasa Indonesia
    country='id',       # Wilayah Indonesia
    sort=Sort.NEWEST,   # Urut dari review terbaru
    count=10000         # Jumlah review yang diambil
)

# Simpan hanya kolom review & rating
df = pd.DataFrame(result)[['content', 'score']]
df.columns = ['review', 'rating']

df.to_csv('ulasan_byond_10.000.csv', index=False, encoding='utf-8-sig')
print("✅ Data berhasil diunduh ke 'ulasan_byond_10.000.csv'")


✅ Data berhasil diunduh ke 'ulasan_byond_10.000.csv'


In [ ]:
import pandas as pd
import re

# Baca dataset
df = pd.read_csv("ulasan_byond_10.000.csv")

# Kamus kata kunci
positive_words = ["bagus", "mantap", "puas", "baik", "cepat", "nyaman", "mudah", "suka", "membantu", "keren", "responsif", "praktis", "sempurna", "aman", "lancar", "terbaik", "stabil", "hebat",  ]
negative_words = ["buruk", "error", "kecewa", "lambat", "jelek", "parah", "tidak bisa", "macet", "hilang", "sulit", "gangguan", "lemot", "kecewa", "ribet", "masalah", "bug", "parah"]

# Fungsi deteksi kata kunci
def detect_sentiment(text):
    text = str(text).lower()
    if any(re.search(rf"\b{kw}\b", text) for kw in positive_words):
        return "positif"
    elif any(re.search(rf"\b{kw}\b", text) for kw in negative_words):
        return "negatif"
    else:
        return None

# Label awal berdasarkan kamus
df["label_kamus"] = df["review"].apply(detect_sentiment)

# Perkuat label dengan rating
def label_from_rating(rating):
    if rating >= 4:
        return "positif"
    elif rating <= 3:
        return "negatif"
    else:
        return None

df["label_rating"] = df["rating"].apply(label_from_rating)

# Gabungkan label
def combine_labels(row):
    if row["label_kamus"]:
        return row["label_kamus"]  # Jika ada kata kunci, pakai itu
    else:
        return row["label_rating"] # Kalau tidak ada, pakai label rating

df["final_label"] = df.apply(combine_labels, axis=1)

# Simpan hasil
df.to_csv("ulasan_byond_labeled_10.000.csv", index=False, encoding="utf-8-sig")

print("✅ Pelabelan selesai! File tersimpan sebagai 'ulasan_byond_labeled_10.000.csv'")


✅ Pelabelan selesai! File tersimpan sebagai 'ulasan_byond_labeled_10.000.csv'
